In [ ]:
## Create a model instance and stream the response from the model.

from langchain_ollama import OllamaLLM

model = OllamaLLM(model="mistral")

chunks = []
for chunk in model.stream("what color is the sky"):
    chunks.append(chunk)
    print(chunk, end="", flush=True)

In [ ]:
#Use a string parser to parse the streamed output from the model

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("tell me a joke about {topic}")

chain = prompt | model | StrOutputParser()

for chunk in chain.stream({"topic": "Cricket"}):
    print(chunk, end="", flush=True)

In [ ]:
# Use a JSON parser to parse the streamed output from the model
# The JSON parser will try to create a valid JSON with each chunk streamed by the model.

from langchain_core.output_parsers import JsonOutputParser

chain = (
    model | JsonOutputParser()
)

query =  """output a list of the countries france, spain and japan and their populations in JSON format. 
            'Use a dict with an outer key of "countries" which contains a list of countries. '
            "Each country should have the key `name` and `population`"""

async for text in chain.astream(query):
    print(text, flush=True)

In [56]:
# Method to custom parse the streamed response from the model

def extract_country_names(inputs):
    """
    Method to extract the names of the countries from the model response.
    """
    if not isinstance(inputs, dict):
        return ""
    if "countries" not in inputs:
        return ""
    countries = inputs["countries"]
    if not isinstance(countries, list):
        return ""
    country_names = [country.get("name") for country in countries if isinstance(country, dict)]
    return country_names

In [ ]:
#Use the custom parser to extract the country names from the model response

chain = model | JsonOutputParser() | extract_country_names
async for text in chain.astream(query):
    print(text, end="|", flush=True)